In [1]:
pip install ollama streamlit nest_asyncio pyngrok

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ollama
import nest_asyncio

In [2]:
# Hey Jupyter kernel, behave nicely if someone tries to start another async loop inside you.
# manage clashes of the kernal, when streamlit is running
# event loop management
nest_asyncio.apply()

In [3]:
def analyze_customer_email(email: str) -> str:
    """
    Step 1: Analyze and extract key info (intent, order number, sentiment, etc.)
    """
    extract_prompt = f"""
    Analyze the following customer email.
    1. Classify the intent (refund request, delivery status, product inquiry, complaint, etc.)
    2. Extract key details such as order number, product, sentiment, urgency.
    3. Return the result as JSON with keys: intent, order_number, product, sentiment, urgency, summary.

    Customer Email:
    {email}
    """
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": extract_prompt}]
    )
    return response['message']['content']

In [4]:
def draft_reply(email: str, analysis: str) -> str:
    """
    Step 2: Generate support reply with empathy, professionalism, and business alignment
    """
    business_context = """
    You are a professional customer support representative at a consumer electronics company. 
    Always:
    - Be polite, empathetic, and concise. 
    - Acknowledge the customer situation.
    - Include relevant details (order info, delivery timing, etc.) when available.
    - End positively, reassuring the customer that we value them.
    """

    reply_prompt = f"""
    {business_context}

    The customer sent the following email:
    {email}

    Analysis of their message:
    {analysis}

    Draft a polite, appropriate email response:
    """
# "role": "user" means: treat this as if it came from the “user” in a chat.
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": reply_prompt}]
    )
    return response['message']['content']
# =====================================
# 4️⃣ Test the pipeline directly (without UI)
# =====================================
customer_email = """
Hi Support,
I ordered a Bluetooth speaker last week (Order #12345), but it still hasn’t arrived. 
Can you please tell me when it will be delivered? I needed it for a birthday gift.
"""

analysis = analyze_customer_email(customer_email)
reply = draft_reply(customer_email, analysis)

print("🔍 Extracted Understanding:\n", analysis, "\n")
print("✉️ Draft Reply:\n", reply)

🔍 Extracted Understanding:
 ```json

{

  "intent": "delivery status inquiry",

  "order_number": "12345",
  "product": "Bluetooth speaker",

  "sentiment": "concerned/anxious",

  "urgency": "high, due to the birthday gift context"

}

```

Summary: The customer is inquiring about the delivery status of a Bluetooth speaker (Order #12345) that has not yet arrived. They express concern because they intended it as a birthday present and are awaiting an urgent update on its estimated arrival time. 

✉️ Draft Reply:
 
Dear [Customer's Name],


Thank you for reaching out to us regarding your Bluetooth speaker order (Order #12345). We understand how important it is for gifts like yours arrive on time and we apologize for any inconvenience caused by this delay. I have checked our latest tracking information, and as of now, the package was last located in transit with a delivery estimate to be updated within 2-3 hours based on its current location.


We appreciate your patience while sorting o

In [10]:
%%writefile support_ai_app.py
import streamlit as st
import ollama

def analyze_customer_email(email: str) -> str:
    extract_prompt = f"""
    Analyze the following customer email.
    1. Classify the intent (refund request, delivery status, product inquiry, complaint, etc.)
    2. Extract key details such as order number, product, sentiment, urgency.
    3. Return the result as JSON with keys: intent, order_number, product, sentiment, urgency, summary.

    Customer Email:
    {email}
    """
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": extract_prompt}]
    )
    return response['message']['content']


def draft_reply(email: str, analysis: str) -> str:
    business_context = """
    You are a professional customer support representative at a consumer electronics company.
    Always:
    - Be polite, empathetic, and concise.
    - Acknowledge the customer situation.
    - Include relevant details (order info, delivery timing, etc.) when available.
    - End positively, reassuring the customer that we value them.
    """

    reply_prompt = f"""
    {business_context}

    The customer sent the following email:
    {email}

    Analysis of their message:
    {analysis}

    Draft a polite, appropriate email response:
    """

    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": reply_prompt}]
    )
    return response['message']['content']


# ------------------ UI -------------------------
st.set_page_config(page_title="AI Customer Support Assistant", page_icon="📩")

st.title("📩 AI-Powered Customer Support Assistant")
st.write("Paste a customer email and let AI analyze and draft a polite, professional reply.")

# Input from user
customer_email = st.text_area("Customer Email:", height=200, placeholder="Paste the customer email here...")

if st.button("Generate Reply"):
    if customer_email.strip():
        with st.spinner("🔍 Analyzing email..."):
            analysis = analyze_customer_email(customer_email)
        st.subheader("Extracted Understanding")
        st.json(analysis)

        with st.spinner("✍️ Drafting reply..."):
            reply = draft_reply(customer_email, analysis)

        st.subheader("AI Drafted Reply")
        st.write(reply)
    else:
        st.warning("Please paste a customer email first.")

Overwriting support_ai_app.py


In [11]:
!streamlit run support_ai_app.py

^C


In [ ]:
##!streamlit run support_ai_app.py --server.headless true --server.port 8501